In [1]:
import os

In [38]:
%%bash 

# Input Arguments
# Example of input arguments for the model analysis component
export PROJECT="irn-70656-dev-1307100302"
export REGION='europe-west1'
export BUCKET="bike-sharing-pipeline-metadata"
export PIPELINE_VERSION="v0_1"
export DATA_VERSION="200903_214338"
export MODEL_VERSION="200904_221959"
export TRIAL_ID="None"
export TF_VERSION='2.2'
export PYTHON_VERSION='3.7'
export DEPLOYMENT_FLAG='OK'

# Setting Paths 
# Set up some globals for gcs file
HANDLER='gs://' # ../ for local data, gs:// for cloud data

BASE_DIR=${HANDLER}${BUCKET}/${PIPELINE_VERSION}
RUN_DIR=${BASE_DIR}/run/${DATA_VERSION}

if [ $TRIAL_ID == 'None' ]; then
    MODEL_DIR=${RUN_DIR}/model_training/${MODEL_VERSION}/export/exporter
else 
    MODEL_DIR=${RUN_DIR}/model_training/${MODEL_VERSION}/${TRIAL_ID}/export/exporter
fi

# Setting Model Name and Version on AI Platform

MODEL_NAME="bike_sharing_model"
MODEL_VERSION_NAME=${PIPELINE_VERSION}__${DATA_VERSION}__${MODEL_VERSION}

for EXPORTER in $(gsutil ls  ${MODEL_DIR}); do 
    EXPORTER_DIR=$EXPORTER
done; 

# Deploy Model if deployment flag is 'OK'
# Deploy best model from hyperparameter tuning
if [ $DEPLOYMENT_FLAG == 'OK' ]; then
    if gcloud ai-platform models create ${MODEL_NAME} --regions ${REGION}; then
        echo "INFO: A new model ${MODEL_NAME} has been created on AI Platform."
    else
        echo "INFO: A model with same name already exists on AI Platform."
    fi
    if gcloud ai-platform versions create ${MODEL_VERSION_NAME} --model ${MODEL_NAME} --origin ${EXPORTER_DIR} --runtime-version ${TF_VERSION} --python-version ${PYTHON_VERSION}; then
        echo "INFO: A new version ${MODEL_VERSION_NAME} has been deployed for model ${MODEL_NAME}."
        gcloud ai-platform versions set-default ${MODEL_VERSION_NAME} --model=${MODEL_NAME} 
        echo "INFO: The new version ${MODEL_VERSION_NAME} has been set as default version for model ${MODEL_NAME}."
    
    # Update pipeline frozen statistics and schema
    DATA_SCHEMA_DIR=${RUN_DIR}/data_validation/schema/data_schema.txt
    FREEZE_SCHEMA_DIR=gs://${BUCKET}/${PIPELINE_VERSION}/freeze/frozen_schema.txt

    # Update pipeline frozen statistics and schema
    DATA_STATS_DIR=${RUN_DIR}/data_validation/stats/data_stats.txt
    FREEZE_STATS_DIR=gs://${BUCKET}/${PIPELINE_VERSION}/freeze/frozen_stats.txt

    gsutil cp ${DATA_SCHEMA_DIR} ${FREEZE_SCHEMA_DIR}
    gsutil cp ${DATA_STATS_DIR} ${FREEZE_STATS_DIR}
    
    echo "INFO: Pipeline frozen statistics and schema were updated."

    else
        echo "INFO: A version with same name exists. This version has already been deployed and will not be deployed again."
    fi

else
    echo "INFO: The new model does not improve performance and was not deployed."
fi

# Write output file for next step in pipeline

echo "INFO: Done."

# Update pipeline frozen statistics and schema
DATA_SCHEMA_DIR=${RUN_DIR}/data_validation/schema/data_schema.txt
FREEZE_SCHEMA_DIR=gs://${BUCKET}/${PIPELINE_VERSION}/freeze/frozen_schema.txt

# Update pipeline frozen statistics and schema
DATA_STATS_DIR=${RUN_DIR}/data_validation/stats/data_stats.txt
FREEZE_STATS_DIR=gs://${BUCKET}/${PIPELINE_VERSION}/freeze/frozen_stats.txt

gsutil cp ${DATA_SCHEMA_DIR} ${FREEZE_SCHEMA_DIR}
gsutil cp ${DATA_STATS_DIR} ${FREEZE_STATS_DIR}

INFO: A model with same name already exists on AI Platform.
INFO: A version with same name exists. This version has already been deployed and will not be deployed again.
INFO: Done.


Using endpoint [https://ml.googleapis.com/]
ERROR: (gcloud.ai-platform.models.create) Resource in project [irn-70656-dev-1307100302] is the subject of a conflict: Field: model.name Error: A model with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A model with the same name already exists.
    field: model.name
Using endpoint [https://ml.googleapis.com/]
ERROR: (gcloud.ai-platform.versions.create) ALREADY_EXISTS: Field: version.name Error: A version with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A version with the same name already exists.
    field: version.name
Copying gs://bike-sharing-pipeline-metadata/v0_1/run/200903_214338/data_validation/schema/data_schema.txt...
/ [1 files][  2.0 KiB/  2.0 KiB]                                                
Operation completed over 1 objects/2.0 KiB.                                      
Copying g